In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable  
import torch.utils.data as Data  
import time
import pickle
import numpy as np

In [7]:
EPOCH = 1  
BATCH_SIZE = 50  
LR = 3e-4

def grid_arr(arr):
    ret = np.zeros(shape=(4,4)+(16,),dtype =bool)
    for r in range(4):
        for c in range(4):
            ret[r, c, map_table[arr[r, c]]] = 1
    return ret

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( #input shape (4,4,16)
                        nn.Conv2d(in_channels=16, #input height 
                        out_channels=60, #n_filter
                        kernel_size=(4,1), #filter size
                        stride=1, #filter step
                        padding=1 #con2d出来的图片大小不变
                        ), #output shape (60, 3, 6)
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=1) 
             
           )
        self.conv2 = nn.Sequential(nn.Conv2d(60, 60, (1,4), 1, 1), #output shape (60, 4, 4)
                                    nn.ReLU(),
                                     nn.MaxPool2d(1))
        self.conv3 = nn.Sequential(nn.Conv2d(60, 60, (2,2), 1, 0), #output shape (60, 3, 3)
                                    nn.ReLU(),
                                     nn.MaxPool2d(1))
        self.conv4 = nn.Sequential(nn.Conv2d(60, 60, (3,3), 1, 1), #output shape (60, 3, 3)
                                    nn.ReLU(),
                                     nn.MaxPool2d(1))
        self.conv5 = nn.Sequential(nn.Conv2d(60, 60, (4,4), 1, 1), #output shape (60, 2, 2)
                                    nn.ReLU(),
                                     nn.MaxPool2d(1))
        self.out = nn.Linear(60*4*4,4)
       
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), -1)  #flat (batch_size, 32*7*7)
        output = self.out(x)
        return output


cnn = CNN()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)  
loss_function = nn.CrossEntropyLoss()


pick_file = open('training_data.pkl', 'rb')
dataset = pickle.load(pick_file)
data_x = []
data_y = []
for i in len(dataset):
    data_x.append(dataset[i][0])
    data_y.append(dataset[i][1])

for epoch in range(EPOCH):  
    start = time.time() 
    for i in range(len(dataset)):  
        b_x = Variable(grid_arr(data_x), requires_grad=False) 
        b_y = Variable(data_y, requires_grad=False)  
  
        output = cnn(b_x)  
        loss = loss_function(output, b_y)  
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()  
  
        if step % 100 == 0:  
            print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss.data[0])  
    duration = time.time() - start 
    print('Training duation: %.4f'%duration)



In [9]:
help(Data.dataloader)

Help on module torch.utils.data.dataloader in torch.utils.data:

NAME
    torch.utils.data.dataloader

CLASSES
    builtins.object
        DataLoader
        ExceptionWrapper
        ManagerWatchdog
    
    class DataLoader(builtins.object)
     |  Data loader. Combines a dataset and a sampler, and provides
     |  single- or multi-process iterators over the dataset.
     |  
     |  Arguments:
     |      dataset (Dataset): dataset from which to load the data.
     |      batch_size (int, optional): how many samples per batch to load
     |          (default: ``1``).
     |      shuffle (bool, optional): set to ``True`` to have the data reshuffled
     |          at every epoch (default: ``False``).
     |      sampler (Sampler, optional): defines the strategy to draw samples from
     |          the dataset. If specified, ``shuffle`` must be False.
     |      batch_sampler (Sampler, optional): like sampler, but returns a batch of
     |          indices at a time. Mutually exclusiv